In [9]:
import robotic as ry
import time
with open("logger.txt","w") as f:
    f.write("")
try:
    del C
    del bot
except:
    pass

C = ry.Config()
# C.watchFile("throwing1.g")
C.addFile("throwing_bare.g")
qHome = C.getJointState()
C.view()
#time.sleep(30)
C.getFrame('cargo').unLink()
C.addFrame("initial").setPosition([-0.6,2.5,0.3]).setShape(ry.ST.marker,[.3])
C.addFrame("throw").setPosition([0,2,1.2]).setShape(ry.ST.marker,[.3])
C.view()

0

In [6]:
def pre_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) 
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)

    komo.addObjective([1., 3.], ry.FS.positionRel, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0, 0, 0])  

    # komo.addObjective([1., 3.], ry.FS.scalarProductXX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductYX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e2], [1]) 
    # komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 

    komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [grasp_frame_name, gripper_name], ry.OT.eq, [1e1], [1]) 


    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo
qHome = C.getJointState()
def post_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 1, 1, 0, True)
    komo.addObjective([], ry.FS.positionDiff, ["l_gripper","initial"], ry.OT.eq, [1e1], [0,0,0])
    komo.addObjective([], ry.FS.scalarProductYZ, ["l_gripper","initial"], ry.OT.eq, [1e1], [-1])
    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo
q0 = C.getJointState()
komo0 = pre_grasp_komo(C, 'l_gripper', 'cargo', q0, qHome)
path0 = komo0.getPath()

komo = post_grasp_komo(C, 'l_gripper', 'cargo', q0, qHome)
path = komo.getPath()
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)
# print(path.shape)
bot.move(path0, [1.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

#bot.gripperCloseGrasp(ry._left, 'cargo',speed=1000000000000,width=0.04)
bot.gripperClose(ry._left,width=0.06) 
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)
bot.move(path, [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)
#C.getFrame('cargo').unLink()
C.attach('l_gripper', 'cargo')

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
{ time: 0.01227, evals: 27, done: 1, feasible: 1, sos: 0.206135, f: 0, ineq: 0, eq: 0.00248478 }
----newton---- initial point f(x):17270.3 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    11713.8  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    6805.72  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    3626.02  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):     2024.4  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    1019.56  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):     396.06  ACCEPT
--newton-- it:   7  |Delta|:        0.2  alpha:          1  evals:   8  f(y):    76.0648  ACCEPT
--newton-- it:   8  |Delta|:   0.174241  alpha:          1  evals:   9  f(y

In [7]:
#code for calculating velocity
def vel_komo():
    q0 = C.getJointState()
    komo = ry.KOMO(C, 1, 1, 1, True)
    komo.addObjective([], ry.FS.position, ["l_gripper"], ry.OT.eq, [1e-1], [4.2,-3.5,4],1)
    komo.addObjective([],ry.FS.positionDiff,["l_gripper","throw"],ry.OT.sos,[1e0],[0,0,0])
    ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    #komo.addObjective([1.],ry.FS.scalarProductXZ,["base","l_gripper"],ry.OT.eq,[1e1],[-1],0)
    #print(komo.report())
    print(f"ret!!!: {ret2}")
    return komo
komo = vel_komo()
# bot = ry.BotOp(C,False)
# time.sleep(5)
path = komo.getPath()
print(f"path size: {path.size}")
bot.move(path,[1.])
time.sleep(0.55)
#print(f"bot end time:{bot.getTimeToEnd()}")
bot.gripperMove(ry._left,width=1)
# while bot.gripperDone(ry._left):
#     bot.sync(C,.1)
print(f"bot end time:{bot.getTimeToEnd()}")
while bot.getTimeToEnd()>0:
    bot.sync(C,.1)


====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):1.84656 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    1.39157  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    1.04791  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):   0.791769  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):   0.603585  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):   0.470295  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):   0.384296  ACCEPT
--newton-- it:   7  |Delta|:   0.133697  alpha:          1  evals:   8  f(y):   0.349428  ACCEPT
--newton-- it:   8  |Delta|:  0.0595521  alpha:          1  evals:   9  f(y):   0.337307  ACCEPT
--newton-- it:   9  |Delta|:  0.0477899  alpha:          1  evals:  10  f(y

bot end time:0.9200000000000017
-- kin_physx.cpp:removeJoint:891(0) REMOVING JOINT 0x5b9450ab88e0-0x5b944e3dee80 of type rigid


In [8]:
time.sleep(4)
del C
del bot

-- bot.cpp:~BotOp:135(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:57(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
